<a href="https://colab.research.google.com/github/Nooraldin2001/LLM_PRACTICES/blob/main/embeddings_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch10/setup.py')

%run -i setup.py

You are working on Google Colab.
Files will be downloaded to "/content".
!wget -P /content https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/settings.py
!wget -P /content/packages/blueprints https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/packages/blueprints/__init__.py
!wget -P /content/packages/blueprints https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/packages/blueprints/embeddings.py
!wget -P /content/data/reddit-selfposts https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/data/reddit-selfposts/reddit-selfposts-ch10.db
!wget -P /content/ch10 https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/ch10/requirements.txt

Additional setup ...
!pip install -r /content/ch10/requirements.txt
!mkdir -p models


In [2]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

%precision 3
np.set_printoptions(suppress=True)

sys.path.append(BASE_DIR + '/packages')


In [3]:
import os
os.environ['GENSIM_DATA_DIR'] = './models'

In [4]:
pd.options.display.float_format = '{:.0f}'.format

In [5]:
import gensim.downloader as api

info_df = pd.DataFrame.from_dict(api.info()['models'], orient='index')
info_df[['file_size', 'base_dataset', 'parameters']].head(5)

,file_size,base_dataset,parameters
fasttext-wiki-news-subwords-300,1005007116,"Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens)",{'dimension': 300}
conceptnet-numberbatch-17-06-300,1225497562,"ConceptNet, word2vec, GloVe, and OpenSubtitles 2016",{'dimension': 300}
word2vec-ruscorpora-300,208427381,Russian National Corpus (about 250M words),"{'dimension': 300, 'window_size': 10}"
word2vec-google-news-300,1743563840,Google News (about 100 billion words),{'dimension': 300}
glove-wiki-gigaword-50,69182535,"Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)",{'dimension': 50}


In [6]:
info_df.head(3)

,num_records,file_size,base_dataset,reader_code,license,parameters,description,read_more,checksum,file_name,parts,preprocessing
fasttext-wiki-news-subwords-300,999999,1005007116,"Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens)",https://github.com/RaRe-Technologies/gensim-data/releases/download/fasttext-wiki-news-subwords-300/__init__.py,https://creativecommons.org/licenses/by-sa/3.0/,{'dimension': 300},"1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).","[https://fasttext.cc/docs/en/english-vectors.html, https://arxiv.org/abs/1712.09405, https://arxiv.org/abs/1607.01759]",de2bb3a20c46ce65c9c131e1ad9a77af,fasttext-wiki-news-subwords-300.gz,1,NaN
conceptnet-numberbatch-17-06-300,1917247,1225497562,"ConceptNet, word2vec, GloVe, and OpenSubtitles 2016",https://github.com/RaRe-Technologies/gensim-data/releases/download/conceptnet-numberbatch-17-06-300/__init__.py,https://github.com/commonsense/conceptnet-numberbatch/blob/master/LICENSE.txt,{'dimension': 300},ConceptNet Numberbatch consists of state-of-the-art semantic vectors (also known as word embeddings) that can be used directly as a representation of word meanings or as a starting point for furth...,"[http://aaai.org/ocs/index.php/AAAI/AAAI17/paper/view/14972, https://github.com/commonsense/conceptnet-numberbatch, http://conceptnet.io/]",fd642d457adcd0ea94da0cd21b150847,conceptnet-numberbatch-17-06-300.gz,1,NaN
word2vec-ruscorpora-300,184973,208427381,Russian National Corpus (about 250M words),https://github.com/RaRe-Technologies/gensim-data/releases/download/word2vec-ruscorpora-300/__init__.py,https://creativecommons.org/licenses/by/4.0/deed.en,"{'dimension': 300, 'window_size': 10}",Word2vec Continuous Skipgram vectors trained on full Russian National Corpus (about 250M words). The model contains 185K words.,"[https://www.academia.edu/24306935/WebVectors_a_Toolkit_for_Building_Web_Interfaces_for_Vector_Semantic_Models, http://rusvectores.org/en/, https://github.com/RaRe-Technologies/gensim-data/issues/3]",9bdebdc8ae6d17d20839dd9b5af10bc4,word2vec-ruscorpora-300.gz,1,The corpus was lemmatized and tagged with Universal PoS


In [7]:
pd.options.display.float_format = '{:.2f}'.format

In [8]:
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [9]:
%precision 2

'%.2f'

In [10]:
v_king = model['king']
v_queen = model['queen']

print("vector size:", model.vector_size)
print("v_king:", v_king[:10])
print("v_queen:", v_queen[:10])
print("similarity:", model.similarity("king", "queen"))

vector size: 50
v_king: [ 0.5   0.69 -0.6  -0.02  0.6  -0.13 -0.09  0.47 -0.62 -0.31]
v_queen: [ 0.38  1.82 -1.26 -0.1   0.36  0.6  -0.18  0.84 -0.06 -0.76]
similarity: 0.78390425


In [11]:
%precision 3

'%.3f'

In [12]:
model.most_similar("king", topn=3)

[('prince', 0.824), ('queen', 0.784), ('ii', 0.775)]

In [13]:
v_lion = model['lion']
v_nano = model['nano']

#Compute cosine similarities between one vector and a set of other vectors.
model.cosine_similarities(v_king, [v_lion, v_nano, v_queen])

array([0.478, 0.051, 0.784], dtype=float32)

In [14]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)

[('queen', 0.852), ('throne', 0.766), ('prince', 0.759)]

In [15]:
model.most_similar(positive=['paris', 'germany'], negative=['berlin'], topn=3)

[('france', 0.927), ('french', 0.832), ('belgium', 0.809)]

In [16]:
model.most_similar(positive=['france', 'berlin'], negative=['paris'], topn=3)

[('germany', 0.921), ('denmark', 0.808), ('poland', 0.794)]

In [17]:
model.most_similar(positive=['france', 'capital'], topn=3)

[('paris', 0.784), ('french', 0.780), ('europe', 0.771)]

In [18]:
model.most_similar(positive=['greece', 'capital'], topn=3)

[('central', 0.797), ('western', 0.757), ('region', 0.750)]

In [19]:
#data preparation

db_name = 'reddit-selfposts.db'
db_name = f"{BASE_DIR}/data/reddit-selfposts/reddit-selfposts-ch10.db" ### real location
con = sqlite3.connect(db_name)
df = pd.read_sql("SELECT subreddit, lemmas, text from posts_nlp", con)
con.close()

df['lemmas'] = df['lemmas'].str.lower().str.split() #lower case and splitting is usually a way to preprocess and tokinize the text dataset
sents = df['lemmas'] # training data

In [20]:
# phrases
from gensim.models.phrases import Phrases, npmi_scorer
import gensim

"""
The compatibility issue in this code arises because of differences in
how Gensim 3.x and Gensim 4.x handle the delimiter parameter in the Phrases class.
"""

# solved compatibility issue for Gensim 4.x
if gensim.__version__[0] > '3': # gensim 4.x string delimiter
    delim = '-'
else: # gensim 3.x - byte delimiter
    delim = b'-'

phrases = Phrases(sents, min_count=10, threshold=0.3,
                  delimiter=delim, scoring=npmi_scorer)

In [21]:
sent = "I had to replace the timing belt in my mercedes c300".split()
phrased = phrases[sent]
print(phrased)

['I', 'had', 'to', 'replace', 'the', 'timing-belt', 'in', 'my', 'mercedes-c300']


In [24]:
if gensim.__version__[0] > '3': # gensim 4.x string delimiter

    phrase_df = pd.DataFrame(phrases.find_phrases(sents),
                             columns=['phrase', 'score'])

    phrase_df = pd.DataFrame.from_dict(phrases.find_phrases(sents),
                                       orient='index').reset_index()

    phrase_df.columns = ['phrase', 'score']
    phrase_df = phrase_df[['phrase', 'score']].drop_duplicates()
    phrase_df = phrase_df.sort_values(by='score', ascending=False).reset_index(drop=True)

else: # gensim 3.x - byte delimiter
    phrase_df = pd.DataFrame(phrases.export_phrases(sents, out_delimiter=delim),
                          columns=['phrase', 'score'])
    phrase_df = phrase_df[['phrase', 'score']].drop_duplicates()
    phrase_df = phrase_df.sort_values(by='score', ascending=False).reset_index(drop=True)
    phrase_df['phrase'] = phrase_df['phrase'].map(lambda p: p.decode('utf-8'))

In [27]:
phrase_df[phrase_df['phrase'].str.contains('mercedes')].head(3)

,phrase,score
83,mercedes-benz,0.80
1416,mercedes-c300,0.47


In [32]:
phrase_df.query("score > 0.7").sample(100)

,phrase,score
4,forty-eight,1.00
218,wiring-harness,0.71
87,king-ranch,0.79
177,focus-st,0.73
90,license-plate,0.78
...,...,...
222,v-rod,0.71
189,tread-depth,0.72
77,boss-302,0.80
34,lithium-ion,0.91


## Blueprint: Training Models with Gensim

In [33]:

logging.getLogger().setLevel(logging.WARNING)
"""
This means the logger will only handle messages at the WARNING level or higher.
Messages with lower severity (like INFO or DEBUG) will be ignored.
"""
sents = df['lemmas']
phrases = Phrases(sents, min_count=10, threshold=0.7,
                  delimiter=delim, scoring=npmi_scorer)
#
df['phrased_lemmas'] = df['lemmas'].progress_map(lambda s: phrases[s])
sents = df['phrased_lemmas']

  0%|          | 0/20000 [00:00<?, ?it/s]

In [34]:
sents.head()

,phrased_lemmas
0,"[club, color, funny, story, go, to, college, in, las-vegas, be, before, know, about, motorcycle, whatsoever, and, college, buddy, would, always, go, out, on, strip, to, dance, club, always, end, u..."
1,"[roush, vs, shleby, gt500, be, try, determine, be, fast, and, have, see, dealership, video, with, two, racing(roush, win, 2/3, but, be, wonder, if, be, just, because, of, bigass, supercharg, in, r..."
2,"[2001, golf, wagon, look, for, insight, hello, try, find, information, on, replace, 2001, golf, wagon, starter, gas, be, go, out, and, go, to, dealership, be, quite, out, of, range, right, now, no..."
3,"[be, 250, coolant, flush, change, _url_, have, is, 250, awd, from, 2006, about, 73, k, mile, on, have, never, touch, engine, radiator, coolant, and, can, find, on, when, change, in, book, just, sa..."
4,"[gen1, mpg, w/, dead, battery, hi, new, to, subreddit, be, consider, buy, gen1, volt, but, can, find, straight, answer, as, to, kind, of, mpg, get, after, battery, be, completely, dead, say, take,..."


In [35]:
from gensim.models import Word2Vec

model = Word2Vec(sents,
                 vector_size=300,
                 window=5,
                 sg=1,
                 negative=5,
                 min_count=1,
                 workers=4,
                 epochs=5)

In [36]:
logging.getLogger().setLevel(logging.ERROR)

In [37]:
model.save('./models/autos_w2v_100_2_full.bin')

In [38]:
from gensim.models import Word2Vec, FastText

model_path = './models'
model_prefix = 'autos'

param_grid = {'w2v': {'variant': ['cbow', 'sg'], 'window': [2,5,30]},
              'ft': {'variant': ['sg'], 'window': [5]}}
size = 100

for algo, params in param_grid.items():
    print(algo) ###
    for variant in params['variant']:
      sg = 1 if variant == 'sg' else 0
      for window in params['window']:
        print(f"  Variant: {variant}, Window: {window}, Size: {size}") ###
        np.random.seed(1) ### to ensure repeatability
        if algo == 'w2v':
          model = Word2Vec(sents,
                           vector_size=size,
                           window=window,
                           sg=sg)
        else:
            model = FastText(sents,
                            vector_size=size,
                            window=window,
                            sg=sg)
        file_name = f"{model_path}/{model_prefix}_{algo}_{size}_{window}_{variant}"
        model.wv.save_word2vec_format(file_name + '.bin', binary=True)


w2v
  Variant: cbow, Window: 2, Size: 100
  Variant: cbow, Window: 5, Size: 100
  Variant: cbow, Window: 30, Size: 100
  Variant: sg, Window: 2, Size: 100
  Variant: sg, Window: 5, Size: 100
  Variant: sg, Window: 30, Size: 100
ft
  Variant: sg, Window: 5, Size: 100


In [39]:
#evaluating different Models
from gensim.models import KeyedVectors
model_path = './models'
names = ['autos_w2v_100_2_cbow', 'autos_w2v_100_2_sg',
         'autos_w2v_100_5_sg', 'autos_w2v_100_30_sg', 'autos_ft_100_5_sg']
models = {}

for name in names:
    file_name = f"{model_path}/{name}.bin"
    print(f"Loading {file_name}")
    models[name] = KeyedVectors.load_word2vec_format(file_name, binary=True)

Loading ./models/autos_w2v_100_2_cbow.bin
Loading ./models/autos_w2v_100_2_sg.bin
Loading ./models/autos_w2v_100_5_sg.bin
Loading ./models/autos_w2v_100_30_sg.bin
Loading ./models/autos_ft_100_5_sg.bin


In [40]:
def compare_models(models, **kwargs):

  df = pd.DataFrame()
  for name, model in models:
    df[name] = [f"{word} {score:.3f}"
      for word, score in model.most_similar(**kwargs)]
  df.index = df.index + 1
  return df

In [46]:
compare_models([(n, models[n]) for n in names], positive='bmw', topn=5).T

,1,2,3,4,5
autos_w2v_100_2_cbow,mercedes 0.867,lexus 0.815,vw 0.792,audi 0.779,volvo 0.774
autos_w2v_100_2_sg,mercedes 0.783,benz 0.704,audi 0.679,mercede 0.679,mercedes-benz 0.673
autos_w2v_100_5_sg,335i 0.766,mercedes 0.750,328i 0.744,135i 0.706,x-drive 0.703
autos_w2v_100_30_sg,328i 0.802,xdrive 0.779,335i 0.774,bimmer 0.748,5-series 0.743
autos_ft_100_5_sg,bmws 0.827,bmwfs 0.771,mercedes_benz 0.770,merc 0.754,m135i 0.753


In [43]:
compare_models([(n, models[n]) for n in names],
               positive=['x3', 'audi'], negative=['bmw'], topn=5).T

,1,2,3,4,5
autos_w2v_100_2_cbow,b8.5 0.848,b7 0.846,b6 0.827,1.8-t 0.824,b8 0.824
autos_w2v_100_2_sg,b9 0.763,sportback 0.752,avant 0.750,b8.5 0.749,tuscon 0.744
autos_w2v_100_5_sg,sportback 0.756,a5 0.752,b8.5 0.748,q5 0.745,a4 0.740
autos_w2v_100_30_sg,a4 0.723,a3 0.703,a6 0.702,sportback 0.682,quattro 0.677
autos_ft_100_5_sg,q5 0.747,s6 0.733,b9 0.730,a4 0.728,b8.5 0.727


In [48]:
compare_models([(n, models[n]) for n in names],
               positive=['spark-plug'], negative=[], topn=5)

,autos_w2v_100_2_cbow,autos_w2v_100_2_sg,autos_w2v_100_5_sg,autos_w2v_100_30_sg,autos_ft_100_5_sg
1,thermostat 0.867,serpentine-belt 0.885,serpentine-belt 0.818,sparkplug 0.789,sparkplug 0.920
2,gasket 0.855,gasket 0.855,plugs 0.794,ngk 0.773,spark 0.878
3,timing-belt 0.847,water-pump 0.839,water-pump 0.791,misfiring 0.755,serpentine-belt 0.833
4,water-pump 0.829,thermostat 0.833,sparkplug 0.786,coil 0.708,bosch 0.817
5,serpentine-belt 0.825,distributor 0.827,ngk 0.783,spark 0.707,ngk 0.810


In [49]:
from umap import UMAP

model = models['autos_w2v_100_30_sg']
words = model.index_to_key # words in vocabulary
wv = [model[word] for word in words] # word vectors

reducer = UMAP(n_neighbors=15, n_components=2, metric='cosine', min_dist=0.1, random_state=12)
embedding = reducer.fit_transform(wv)

In [ ]:
import plotly.express as px
px.defaults.template = 'plotly_white'